In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
print("Hello")

In [ ]:
from sklearn.metrics import mean_squared_log_error
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
import time
import random

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-jul-2021/train.csv')
test = pd.read_csv('../input/tabular-playground-series-jul-2021/test.csv')
sub = pd.read_csv('../input/tabular-playground-series-jul-2021/sample_submission.csv')

In [ ]:
train['date_time'] = pd.to_datetime(train['date_time'], format = "%Y-%m-%d %H:%M:%S")
test['date_time'] = pd.to_datetime(test['date_time'], format= "%Y-%m-%d %H:%M:%S")

In [ ]:
train.info()
test.info()

In [ ]:
def make_new_features(df):
    df["month"] = df["date_time"].dt.month
    df["day_of_week"] = df["date_time"].dt.dayofweek
    df["day_of_year"] = df["date_time"].dt.dayofyear
    df["hour"] = df["date_time"].dt.hour
    df["quarter"] = df["date_time"].dt.quarter
    df["week_of_year"] = df["date_time"].dt.isocalendar().week.astype("int")
    df["is_winter"] = df["month"].isin([1, 2, 12])
    df["is_sprint"] = df["month"].isin([3, 4, 5])
    df["is_summer"] = df["month"].isin([6, 7, 8])
    df["is_autumn"] = df["month"].isin([9, 10, 11])
    df["working_hours"] =  df["hour"].isin(np.arange(8, 21, 1)).astype("int")
    df["is_weekend"] = (train["date_time"].dt.dayofweek >= 5).astype("int")
    return df

In [ ]:
train = make_new_features(train)
test = make_new_features(test)

In [ ]:
train

In [ ]:
test

In [ ]:
train['date_time'] = train['date_time'].astype('datetime64[ns]').astype(np.int64)/10**9
test['date_time'] = test['date_time'].astype('datetime64[ns]').astype(np.int64)/10**9

In [ ]:
train

In [ ]:
test

In [ ]:
columns = test.columns
columns

In [ ]:
columns_train = train.columns
columns_train

In [ ]:
X = train[columns].values
X_test = test[columns].values
target_1 = train['target_carbon_monoxide'].values.reshape(-1,1)
target_2 = train['target_benzene'].values.reshape(-1,1)
target_3 = train['target_nitrogen_oxides'].values.reshape(-1,1)

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV



rf = RandomForestRegressor()
rf.fit(X,target_1)
y_target1_rf = rf.predict(X_test)


rf2 = RandomForestRegressor()
rf2.fit(X,target_2)
y_target2_rf = rf2.predict(X_test)


rf3 = RandomForestRegressor()
rf3.fit(X,target_3)
y_target3_rf = rf3.predict(X_test)

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

gb1= GradientBoostingRegressor()
gb1.fit(X,target_1)
y_target1_gb = gb1.predict(X_test)

gb2 = GradientBoostingRegressor()
gb2.fit(X,target_2)
y_target2_gb = gb2.predict(X_test)

gb3 = GradientBoostingRegressor()
gb3.fit(X,target_3)
y_target3_gb = gb3.predict(X_test)

In [ ]:
sub['target_carbon_monoxide'] = (y_target1_rf + y_target1_gb) / 2
sub['target_benzene'] = (y_target2_rf + y_target2_gb) / 2
sub['target_nitrogen_oxides'] = (y_target3_rf + y_target3_gb) / 2

In [ ]:
sub

In [ ]:
sub.to_csv('submission.csv',index=False)